In [ ]:
# Import library for ReGex, SQLite, and Pandas
import re
import sqlite3
import pandas as pd
import string #tambahan



# Import library for Flask
from flask import Flask, jsonify
from flask import request
from flasgger import Swagger, LazyString, LazyJSONEncoder
from flasgger import swag_from

# Define Swagger UI description
app = Flask(__name__)
app.json_encoder = LazyJSONEncoder
swagger_template = dict(
info = {
    'title': LazyString(lambda: 'API Documentation for Data Processing and Modeling By Fiki Bima'),
    'version': LazyString(lambda: '1.0.0'),
    'description': LazyString(lambda: 'Dokumentasi API untuk Data Processing dan Modeling oleh Fiki Bima'),
    },
    host = LazyString(lambda: request.host)
)
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": 'docs',
            "route": '/docs.json',
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, template=swagger_template,             
                  config=swagger_config)

# Connect to db
conn = sqlite3.connect('data/text.db', check_same_thread=False)

# Define and execute query for create table "data" if not exist
# Table "Data" contains "text" coloumn and "text_clean" coloumn. The two columns have "varchar" data type
conn.execute('''CREATE TABLE IF NOT EXISTS data (text varchar(100), text_clean varchar(100));''')

# Define endpoint for "input teks via form"
@swag_from("docs/text.yml", methods=['POST'])
@app.route('/text', methods=['POST'])
def text_processing():

    # Get text file
    text_dirty = request.form.get('text')
    text = text_dirty.lower()
    
    # Cleansing Process. Remove "Emoticon" and "Punctuation"
    text_fase1 = re.sub(r'[^a-z]', ' ', text)
    text_fase2 = text_fase1.translate(str.maketrans("","",string.punctuation))
    text_clean = text_fase2.strip()
    

    # Define and execute query for insert original text and cleaned text to sqlite database
    conn.execute("INSERT INTO data (text, text_clean) VALUES ('" + text + "', '" + text_clean + "')")
    conn.commit()

    # Define API response
    json_response = {
        'status_code': 200,
        'description': "Text Processed",
        'data': text_clean,
    }
    response_data = jsonify(json_response)
    return response_data
@swag_from("docs/text_data_csv.yml", methods=['POST'])
@app.route("/data/csv",methods=["POST"])
def input_csv():
  if request.method == 'POST':
    
    file = request.files['file']

    try:
      df = pd.read_csv(file,encoding='iso-8859-1')
    except:
      df = pd.read_csv(file, encoding='utf-8')
    (df)
    return "Upload Successfull"

def text_processing_csv():
    
    # Cleansing Process. Remove "Emoticon" and "Punctuation"
    text_fase1_csv = re.sub(r'[^a-z]', ' ', df)
    text_fase2_csv = text_fase1_csv.translate(str.maketrans("","",string.punctuation))
    text_clean_csv = text_fase2_csv.strip()
    
    

    # Define and execute query for insert original text and cleaned text to sqlite database
    conn.execute("INSERT INTO data (text_csv, text_clean_csv) VALUES ('" + text_csv + "', '" + text_clean_csv + "')")
    conn.commit()

    # Define API response
    json_response_csv = {
        'status_code': 200,
        'description': "Text Successfull processed",
        'data': text_clean_csv,
    }
    response_data_csv = jsonify(json_response_csv)
    return response_data_csv

if __name__ == '__main__':
   app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Oct/2022 09:56:18] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Oct/2022 09:56:18] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /docs HTTP/1.1" 308 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /docs/ HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2022 09:56:23] "GET /flasgger_static/favicon-32x32.png HTTP/1.1" 200 -
[2022-10-03 09:56:23,619] ERROR in app: Exception on /docs.json [GET]
Traceback (most recent call last):
  File "C:\Users\HF-User\anaconda3\lib\site-packages\flasgger\utils.py", line 546, in load_from_